Necessary imports

In [45]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [46]:
import os
from PIL import Image
import numpy as np

parent_directory = "/content/drive/MyDrive/hair_type_detector/data"
X = []
y = []

# Define class labels (you can assign based on directory names)
class_labels = {'Straight': 0, 'Wavy': 1, 'curly': 2, 'dreadlocks': 3, 'kinky': 4}

for dir_name in os.listdir(parent_directory):
    dir_path = os.path.join(parent_directory, dir_name)

    if os.path.isdir(dir_path) and dir_name in class_labels:
        print(f"Directory: {dir_name}")

        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)

            if os.path.isfile(file_path):
                try:
                    img = Image.open(file_path)

                    # Convert the image to grayscale (L mode)
                    img = img.convert('L')

                    # Resize the image to a standard size (e.g., 150x150) to make it uniform
                    img = img.resize((150, 150))

                    img_array = np.array(img)
                    X.append(img_array)
                    y.append(class_labels[dir_name])
                except Exception as e:
                    print(f"Error loading image {file_name}: {e}")

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

print(f"Loaded {len(X)} images with {len(y)} labels.")


Directory: Straight
Directory: Wavy
Directory: curly
Directory: dreadlocks
Directory: kinky
Loaded 1992 images with 1992 labels.


Perform data augmentation

In [47]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
print(f"Training set size: {len(X_train)} images")
print(f"Test set size: {len(X_test)} images")

# 2. Data Augmentation for Training Set
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=(0.3, 1.0),
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow(
    X_train,
    y_train,
    batch_size=32,
    subset='training'
)

# Create the generator for the validation set (from training data)
validation_generator = train_datagen.flow(
    X_train,
    y_train,
    batch_size=32,
    subset='validation'
)

# For the actual test set (untouched during training), you can use test_datagen
test_generator = test_datagen.flow(
    X_test,
    y_test,
    batch_size=32
)

Training set size: 1593 images
Test set size: 399 images


In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# Define the model
model = Sequential()

# First convolution layer
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer
model.add(Conv2D(64, (3, 3)))  # Increased the number of filters to 64
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third convolution layer
model.add(Conv2D(128, (3, 3)))  # Further increased to 128 filters
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fourth convolution layer (Newly added)
model.add(Conv2D(256, (3, 3)))  # Adding more filters to capture deeper features
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fifth convolution layer (Newly added)
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output
model.add(Flatten())

# Fully connected layer 1
model.add(Dense(256))  # Increased the number of units for more complexity
model.add(Activation('relu'))

# Dropout to prevent overfitting
model.add(Dropout(0.5))  # Increased dropout rate slightly to prevent overfitting with deeper layers

# Fully connected layer 2
model.add(Dense(128))
model.add(Activation('relu'))

# Dropout to prevent overfitting
model.add(Dropout(0.5))

# Fully connected layer 3
model.add(Dense(64))
model.add(Activation('relu'))

# Dropout to prevent overfitting
model.add(Dropout(0.5))

# Output layer for multiclass classification
model.add(Dense(5))  # 5 output units for 5 classes
model.add(Activation('softmax'))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=100, validation_data=validation_generator, callbacks=[early_stopping])




/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


40/40 ━━━━━━━━━━━━━━━━━━━━ 14s 208ms/step - accuracy: 0.2691 - loss: 1.6080 - val_accuracy: 0.2453 - val_loss: 1.5777
Epoch 2/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.2131 - loss: 1.5978 - val_accuracy: 0.2484 - val_loss: 1.5806
Epoch 3/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - accuracy: 0.2623 - loss: 1.5677 - val_accuracy: 0.2579 - val_loss: 1.5778
Epoch 4/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 109ms/step - accuracy: 0.2334 - loss: 1.5799 - val_accuracy: 0.2484 - val_loss: 1.5774
Epoch 5/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - accuracy: 0.2680 - loss: 1.5755 - val_accuracy: 0.2484 - val_loss: 1.5797
Epoch 6/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 78ms/step - accuracy: 0.2428 - loss: 1.5783 - val_accuracy: 0.2484 - val_loss: 1.5809
Epoch 7/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 114ms/step - accuracy: 0.2474 - loss: 1.5751 - val_accuracy: 0.2484 - val_loss: 1.5817
Epoch 8/100
40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 81ms/step - accuracy: 0.2702 - loss: 1.5693 - val_accuracy: 0.2547 - v

In [49]:
# Print final training and validation accuracy after training
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]

print(f"Final training accuracy: {train_acc}")
print(f"Final validation accuracy: {val_acc}")


Final training accuracy: 0.6180391907691956
Final validation accuracy: 0.5283018946647644
